### Download data and install required libraries 

In [1]:
!git clone https://github.com/stasaki/DEcode.git

Cloning into 'DEcode'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 77 (delta 11), reused 47 (delta 5), pack-reused 6
Unpacking objects: 100% (77/77), done.


In [2]:
%cd DEcode

/content/DEcode


In [16]:
!pip install --upgrade setuptools
!pip2 install shap==0.27.0
!Rscript functions/install_package.R

Requirement already up-to-date: setuptools in /usr/local/lib/python2.7/dist-packages (41.2.0)
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cran.rstudio.com/src/contrib/rjson_0.2.20.tar.gz'
Content type 'application/x-gzip' length 99600 bytes (97 KB)
downloaded 97 KB

* installing *source* package ‘rjson’ ...
** package ‘rjson’ successfully unpacked and MD5 sums checked
** using staged installation
** libs
g++ -std=gnu++11 -I"/usr/share/R/include" -DNDEBUG     -fpic  -g -O2 -fdebug-prefix-map=/build/r-base-uuRxut/r-base-3.6.1=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c dump.cpp -o dump.o
gcc -std=gnu99 -I"/usr/share/R/include" -DNDEBUG     -fpic  -g -O2 -fdebug-prefix-map=/build/r-base-uuRxut/r-base-3.6.1=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c parser.c -o parser.o
gcc -std=gnu99 -I"/usr/share/R/include" -DNDEBUG     -

### Set location of input data

In [0]:
# Set input data

# Gene expression matrix
deg_data_file = "./data/toy/Transcriptome/expdata.txt"

# Location of RNA features
mRNA_data_loc = "./data/toy/RNA_features/"
mRNA_annotation_data = ["POSTAR","TargetScan"]

# Location of promoter features
promoter_data_loc = "./data/toy/Promoter_features/"
promoter_annotation_data = ["GTRD"]

# Genes used for traning, validation, and testing
train_genes = "./data/toy/Gene_splits/train.txt.gz"
validate_genes = "./data/toy/Gene_splits/validate.txt.gz"
test_genes = "./data/toy/Gene_splits/test.txt.gz"    

# Location of hyper-parameter
params_loc='./pretrained/Tissue_gene_params.json'

# Output directory
outloc='./train_out/full_model/'

### Define main function

In [0]:
import os
import sys
sys.path.append('./functions/')
import data
import model_utils
import pandas as pd
import numpy as np
import json

#os.environ["CUDA_VISIBLE_DEVICES"]="0"

def main(params):
    from datetime import datetime
    from keras.callbacks import EarlyStopping, ModelCheckpoint
    from keras.models import load_model
    import numpy as np
    import metrics
    import layer_utils
    import network
    
    print(params)
    just_return_model=False
    
    # model parameters and learning parameters
    max_epoch = 100
    batch_size = 128
    
    # batch initialization
    train_steps, train_batches = data.batch_iter(X_mRNA_train.values[:,1],
                                                 X_promoter_train.values[:,1],
                                                 Y_train.values[:,1:],
                                                 batch_size,
                                                 shuffle=True)
    valid_steps, valid_batches = data.batch_iter(X_mRNA_validate.values[:,1],
                                                 X_promoter_validate.values[:,1],
                                                 Y_validate.values[:,1:],
                                                 batch_size,
                                                 shuffle=True)
    test_steps, test_batches = data.batch_iter(X_mRNA_test.values[:,1],
                                               X_promoter_test.values[:,1],
                                               Y_test.values[:,1:],
                                               batch_size,
                                               shuffle=True)

    # Paramters for network structure
    params['n_feature_mRNA']=X_mRNA_train.values[:,1][0].shape[0]
    params['n_feature_promoter']=X_promoter_train.values[:,1][0].shape[0]
    params['n_out'] = Y_train.values[:,1:].shape[1]
    
    # Define network structure
    model = network.define_network(params)
    
    # If you don't need to traning model and just want to have model structure
    if just_return_model:
        return model
    
    # Set callback functions to early stop training and save the best model so far
    time_stamp=datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    
    callbacks = [EarlyStopping(monitor='val_loss', patience=10),
                    ModelCheckpoint(outloc+time_stamp+'_model.h5', monitor='val_loss', verbose=0,
                    save_best_only=True,
                    save_weights_only=False,
                    mode='min', period=1)]
    
    # Optimizing model
    result = model.fit_generator(train_batches, train_steps, 
                                 epochs=max_epoch,
                                 validation_data=valid_batches,
                                 validation_steps=valid_steps,
                                 callbacks=callbacks,
                                 max_queue_size=10,
                                 verbose=0)
    
    # Test performance
    # Load best model
    model = load_model(outloc+time_stamp+'_model.h5',
                   custom_objects={'pcor': metrics.pcor,
                                  'GlobalSumPooling1D': layer_utils.GlobalSumPooling1D})
    test_performance= np.array(model.evaluate_generator(test_batches,test_steps))
    np.savetxt(outloc+time_stamp+'_test_performance.txt',
               test_performance,delimiter="\t")
    
    # Saving optimization history
    with open(outloc+time_stamp+'_history.json', 'w') as f:
        json.dump(result.history, f)
    
    # Saving model and learning paramters
    with open(outloc+time_stamp+'_params.json', 'w') as f:
        json.dump(params, f)
    
    # Return validation loss for model selection
    validation_loss = np.amin(result.history['val_loss']) 
    
    return {'loss': validation_loss, 'status': STATUS_OK, 'model': model}

### Training model

In [6]:
! mkdir -p "$outloc"
shuffle="None"
# Prepare learning data
Y_train, Y_validate, Y_test, X_mRNA_train, X_mRNA_validate, X_mRNA_test, X_promoter_train, X_promoter_validate, X_promoter_test = data.prep_ml_data_split(
    deg_data_file=deg_data_file,
    mRNA_data_loc=mRNA_data_loc,
    mRNA_annotation_data=mRNA_annotation_data,
    promoter_data_loc=promoter_data_loc,
    promoter_annotation_data=promoter_annotation_data,
    train_genes=train_genes,
    validate_genes=validate_genes,
    test_genes=test_genes,
    outloc=outloc,
    shuffle=shuffle)

# Obtain hyper parameters
with open(params_loc) as f:
    params=json.load(f)  

# Training model
from hyperopt import STATUS_OK
for i in range(10):
    main(params)

# Summarizing the result
! Rscript functions/find_best_model.R "$outloc" &> /dev/null 
with open(outloc+"/summary/best_model.txt") as f:
    best_model=f.readline().rstrip()

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:635: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value
Using TensorFlow backend.
W0826 18:43:00.965044 140387272288128 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_

{u'n_feature_mRNA': 372, u'DNA_n_channel_1st': 120.0, u'Last_fullConLayer': 2.0, u'DNA_conv_strides': 1.0, u'ConvRelu': u'No', u'FullRelu': u'Yes', u'n_feature_mRNA_basic': 1, u'n_feature_promoter': 726, u'lr': 0.001, u'RNA_n_channel_1st': 160.0, u'Last_n_channel': 160.0, u'RNA_conv_strides': 2.0, u'DNA_n_ConvLayer': 4.0, u'RNA_n_ConvLayer': 1.0, u'n_out': 54, u'Add_basic_info': u'No'}


W0826 18:43:01.198169 140387272288128 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0826 18:43:01.309458 140387272288128 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0826 18:43:01.817775 140387272288128 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



{u'n_feature_mRNA': 271, u'DNA_n_channel_1st': 120.0, u'Last_fullConLayer': 2.0, u'DNA_conv_strides': 1.0, u'ConvRelu': u'No', u'FullRelu': u'Yes', u'n_feature_mRNA_basic': 1, u'n_feature_promoter': 601, u'lr': 0.001, u'RNA_n_channel_1st': 160.0, u'Last_n_channel': 160.0, u'RNA_conv_strides': 2.0, u'DNA_n_ConvLayer': 4.0, u'RNA_n_ConvLayer': 1.0, u'n_out': 54, u'Add_basic_info': u'No'}
{u'n_feature_mRNA': 271, u'DNA_n_channel_1st': 120.0, u'Last_fullConLayer': 2.0, u'DNA_conv_strides': 1.0, u'ConvRelu': u'No', u'FullRelu': u'Yes', u'n_feature_mRNA_basic': 1, u'n_feature_promoter': 601, u'lr': 0.001, u'RNA_n_channel_1st': 160.0, u'Last_n_channel': 160.0, u'RNA_conv_strides': 2.0, u'DNA_n_ConvLayer': 4.0, u'RNA_n_ConvLayer': 1.0, u'n_out': 54, u'Add_basic_info': u'No'}
{u'n_feature_mRNA': 271, u'DNA_n_channel_1st': 120.0, u'Last_fullConLayer': 2.0, u'DNA_conv_strides': 1.0, u'ConvRelu': u'No', u'FullRelu': u'Yes', u'n_feature_mRNA_basic': 1, u'n_feature_promoter': 601, u'lr': 0.001, u'RN

IOError: ignored

In [0]:
with open(outloc+"/summary/best_model.txt") as f:
    best_model=f.readline().rstrip()


### Compute Spearman's correlation between actual and predicted expression for each sample

In [0]:
# Prediction for test samples with the best model
model_utils.test_prediction(outloc,
                            best_model,
                            X_mRNA_test,
                            X_promoter_test,
                            Y_test)

In [20]:
! Rscript --vanilla --slave functions/calc_performance.R "$outloc$best_model" &> /dev/null 
pd.read_csv(outloc+best_model+'/test_data/cor_tbl.txt',sep="\t")

,sample,estimate,statistic,p.value,method,alternative
0,Adipose_Subcutaneous,0.139464,2.822948e+06,2.189321e-02,Spearman's rank correlation rho,two.sided
1,Adipose_Visceral_Omentum,0.156453,2.767217e+06,1.003164e-02,Spearman's rank correlation rho,two.sided
2,Adrenal_Gland,0.175140,2.705916e+06,3.891355e-03,Spearman's rank correlation rho,two.sided
3,Artery_Aorta,-0.124397,3.688535e+06,4.110041e-02,Spearman's rank correlation rho,two.sided
4,Artery_Coronary,0.005646,3.261934e+06,9.264274e-01,Spearman's rank correlation rho,two.sided
5,Artery_Tibial,0.278379,2.367244e+06,3.398040e-06,Spearman's rank correlation rho,two.sided
6,Bladder,0.191963,2.650729e+06,1.528781e-03,Spearman's rank correlation rho,two.sided
7,Brain_Amygdala,0.400801,1.965646e+06,7.682070e-12,Spearman's rank correlation rho,two.sided
8,Brain_Anterior_cingulate_cortex_BA24,0.396349,1.980249e+06,1.366575e-11,Spearman's rank correlation rho,two.sided
9,Brain_Caudate_basal_ganglia,0.347193,2.141503e+06,4.577209e-09,Spearman's rank correlation rho,two.sided


### Compute average DeepLIFT score for each regulator

In [21]:
# Estimate variable imporance using test samples
model_utils.compute_DeepLIFT(outloc,
                             best_model,
                             X_mRNA_test,
                             X_promoter_test,
                             Y_test)

0


W0826 18:57:08.636358 140387272288128 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/shap/explainers/deep/deep_tf.py:450: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53


In [22]:
! Rscript --vanilla --slave functions/summarize_DeepLIFT.R "$outloc$best_model" &> /dev/null 
pd.read_csv(outloc+best_model+'/DeepLIFT/RNA_importance_mean.txt',sep="\t")
pd.read_csv(outloc+best_model+'/DeepLIFT/promoter_importance_mean.txt',sep="\t")

,sample_name,AEBP2,AHR,AHRR,APC,AR,ARID1A,ARID1B,ARID2,ARID3A,ARID4B,ARNT,ARNT2,ARNTL,ASCL1,ASCL2,ATF1,ATF2,ATF3,ATF4,ATF5,ATF6,ATF7,ATOH1,ATRX,BACH1,BACH2,BARHL1,BARX1,BARX2,BATF,BATF3,BCL11A,BCL11B,BCL3,BCL6,BHLHE40,BRD4,CDC5L,CDX2,...,ZNF623,ZNF629,ZNF639,ZNF644,ZNF652,ZNF654,ZNF664,ZNF680,ZNF687,ZNF692,ZNF7,ZNF701,ZNF711,ZNF740,ZNF75A,ZNF76,ZNF766,ZNF768,ZNF770,ZNF778,ZNF784,ZNF792,ZNF8,ZNF816,ZNF83,ZNF84,ZNF85,ZNF92,ZSCAN16,ZSCAN2,ZSCAN21,ZSCAN22,ZSCAN29,ZSCAN31,ZSCAN4,ZSCAN5A,ZSCAN5DP,ZSCAN9,ZXDB,ZXDC
0,Adipose_Subcutaneous,-0.000363,0.000006,-0.000423,0.000858,-0.003265,-0.000553,0.003460,-0.000184,-0.001252,-0.003966,0.000239,-0.000085,-0.000257,-0.008232,-0.000301,-0.006879,-0.000121,-0.002783,0.000175,0.000062,1.976943e-04,-0.002148,1.423825e-04,-0.000286,-0.000550,-0.000612,0.000112,0.000159,0.000240,0.002345,-0.000042,0.001864,-0.000866,-0.000063,-0.002421,0.000912,-0.000846,2.758076e-05,0.000029,...,-9.396449e-05,-0.000763,-0.000891,-0.000763,0.001270,-0.000038,-0.000108,-0.000215,0.000496,-0.000584,-0.000477,-0.000605,-0.000519,0.000254,0.000411,-0.000203,-3.950635e-05,0.000140,-0.004199,0.001846,-0.000437,-0.000741,1.752792e-04,1.095512e-05,0.003260,-7.355281e-05,-0.000290,0.000106,0.000325,0.000096,0.000139,0.001557,-0.001071,0.000227,0.000557,0.000622,0.000684,-0.002433,-0.000661,0.000084
1,Adipose_Visceral_Omentum,-0.000449,0.000815,-0.000519,0.001636,-0.045263,-0.000742,0.005242,-0.000572,-0.002177,-0.005168,-0.002727,-0.000141,-0.000523,-0.016472,0.001079,-0.007843,-0.000306,-0.001342,0.000476,0.000045,-7.914809e-05,-0.002528,5.938452e-06,-0.000712,-0.001639,-0.000808,0.000263,0.000679,0.000004,0.003842,-0.000024,0.001725,-0.003723,0.001056,0.002788,0.000241,0.000070,5.800007e-05,-0.002227,...,-2.307161e-05,-0.000903,-0.001340,-0.000873,0.002049,-0.000216,-0.000045,-0.000398,0.000650,-0.000830,-0.000750,-0.001926,-0.000422,0.000604,0.001150,0.000175,3.338502e-06,0.000347,-0.005985,0.004392,-0.000875,-0.000134,1.213426e-04,-3.327525e-05,0.005641,3.335936e-04,-0.000496,0.000298,-0.000327,0.000155,0.000179,-0.003927,-0.001710,0.000143,0.000684,0.000961,0.001147,-0.001658,-0.001541,-0.001554
2,Adrenal_Gland,0.000105,0.001729,0.000204,0.000223,-0.020920,-0.000240,-0.002994,0.000127,0.000375,0.000761,0.000026,0.000015,-0.000286,0.007269,0.001598,0.002153,0.002533,-0.000271,0.000391,0.000027,-2.435988e-04,0.000756,2.199082e-05,0.000039,-0.000274,0.000082,-0.000095,-0.000009,-0.000459,-0.000117,0.000162,0.000734,0.001341,-0.003868,0.001864,0.000858,-0.000236,-9.533766e-07,0.000271,...,-8.314948e-05,-0.000865,0.000884,0.000901,-0.001252,0.000116,-0.000085,0.000120,-0.000061,0.000075,0.000110,0.000430,-0.000345,0.000089,-0.000502,-0.000060,-8.169672e-06,-0.000029,0.001999,-0.000203,-0.000093,-0.000631,2.606687e-04,-9.849179e-05,-0.001557,2.035070e-04,0.000028,-0.000221,-0.000400,0.000199,-0.000101,-0.001964,-0.000084,0.000030,0.000443,-0.000252,-0.000666,-0.000237,0.000037,-0.001543
3,Artery_Aorta,-0.000115,-0.000508,-0.000241,0.001750,-0.017843,-0.000542,0.003975,-0.000293,-0.002374,-0.006113,-0.000084,-0.000155,-0.000054,-0.006948,-0.000090,-0.004004,0.000221,-0.003665,0.000252,-0.000026,-2.786772e-05,-0.001585,1.482681e-04,-0.001254,0.000252,-0.000429,-0.000033,0.000215,0.000208,0.001437,-0.000158,-0.000710,-0.002867,-0.001536,-0.003335,-0.001455,-0.000028,1.412855e-05,-0.001194,...,-9.735100e-05,-0.000466,-0.000113,-0.001041,0.003179,0.000092,-0.000272,-0.000198,0.001865,-0.001000,-0.000689,-0.000180,-0.000933,0.000135,0.000641,-0.000600,1.461708e-06,0.000179,-0.003677,0.002689,-0.000474,-0.001812,1.573903e-04,-8.566207e-06,0.004337,-9.630659e-05,-0.000394,0.000167,-0.000104,0.000117,0.000132,-0.001758,-0.001262,0.000349,0.000784,0.000444,0.000792,-0.002403,-0.000756,0.000149
4,Artery_Coronary,0.000127,-0.000028,-0.000708,0.001644,-0.028052,-0.000725,0.005915,-0.000248,-0.002104,-0.005386,-0.003058,-0.000167,-0.000016,-0.009692,-0.002290,-0.009357,-0.001318,0.000393,0.000199,0.000047,1.405708e-04,-0.002720,6.8280

### Simulate the concequence of regulator knockout

In [0]:
# Simulate the concequence of regulator knockout
genes=["ENSG00000268903","ENSG00000239906"]
model_utils.coexpression_with_KO(genes,
                                 outloc,
                                 best_model,
                                 X_mRNA_test,
                                 X_promoter_test,
                                 Y_test)    

In [24]:
! Rscript --vanilla --slave functions/test_KO.R "$outloc$best_model" &> /dev/null 
pd.read_csv(outloc+best_model+'/regulator_KO/regression_res.txt',sep="\t")

,term,estimate,std.error,statistic,p.value
0,NR2F6,0.005235,0.000191,27.474442,2.954167e-160
1,CEBPA,0.004362,0.000191,22.893958,4.238600e-113
2,EGR1,-0.004215,0.000191,-22.115296,7.657080e-106
3,ESR1,0.003547,0.000190,18.621147,4.186045e-76
4,SP5,-0.003146,0.000191,-16.509360,1.981776e-60
5,ZNF18,0.001565,0.000190,8.218735,2.312262e-16
6,RARA,0.001405,0.000191,7.373096,1.799603e-13
7,RUNX1,0.001354,0.000190,7.108905,1.249829e-12
8,ESRRA,0.001165,0.000190,6.114900,1.002391e-09
9,MYCN,-0.001102,0.000190,-5.786820,7.389296e-09


### Simulate the concequence of binding site removals

In [0]:
# Simulate the concequence of binding site removals
genes=["ENSG00000268903","ENSG00000239906"]
model_utils.coexpression_with_binding_site_removal(genes,
                                                   outloc,
                                                   best_model,
                                                   X_mRNA_test,
                                                   X_promoter_test,
                                                   Y_test)

In [26]:
! Rscript --vanilla --slave functions/test_interval.R "$outloc$best_model" &> /dev/null 
pd.read_csv(outloc+best_model+'/binding_site_removal/regression_res.txt',sep="\t")

,term,estimate,std.error,statistic,p.value,Gene
0,promoter_interval_10,3.881948e-03,5.611975e-05,6.917257e+01,0.000000e+00,ENSG00000268903
1,promoter_interval_12,3.592989e-03,5.606523e-05,6.408587e+01,0.000000e+00,ENSG00000268903
2,promoter_interval_15,6.851069e-03,5.609040e-05,1.221433e+02,0.000000e+00,ENSG00000268903
3,promoter_interval_30,-7.256894e-04,1.711670e-16,-4.239657e+12,0.000000e+00,ENSG00000239906
4,promoter_interval_14,-1.938777e-03,5.608864e-05,-3.456630e+01,2.216499e-247,ENSG00000268903
5,promoter_interval_28,1.410729e-03,5.609633e-05,2.514833e+01,2.308738e-135,ENSG00000268903
6,promoter_interval_27,1.057172e-03,5.607582e-05,1.885255e+01,6.297272e-78,ENSG00000268903
7,promoter_interval_11,5.054476e-04,5.607256e-05,9.014170e+00,2.348348e-19,ENSG00000268903
8,promoter_interval_16,4.797265e-04,5.607375e-05,8.555279e+00,1.348987e-17,ENSG00000268903
9,promoter_interval_29,3.466486e-04,5.608066e-05,6.181250e+00,6.608706e-10,ENSG00000268903
